In [8]:
import numpy as np

import os

# for showing the image
import matplotlib.pyplot as plt

# for testing like opening an image
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [40]:
os.listdir('./train/')

['.DS_Store', 'healthy', 'cordana', 'pestalotiopsis', 'sigatoka']

In [22]:
# training dataset path
training_dataset_path = './train/'
# path to each dataset
healthy_banana_dataset_path = './train/healthy/'
cordona_banana_dataset_path = './train/cordana/'
pestalotiopsis_banana_dataset_path = './train/pestalotiopsis/'
sigatoka_banana_dataset_path = './train/sigatoka/'

# Find mean and std

In [41]:
training_transforms = transforms.Compose([transforms.ToTensor()]) # convert images to pytorch tensors

In [42]:
train_dataset = torchvision.datasets.ImageFolder(root=training_dataset_path, transform=training_transforms)

In [43]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

In [44]:
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_images = 0

    for images, _ in loader:
        images_per_batch = images.size(0)
        # print(images.shape) 
        images = images.view(images_per_batch, images.size(1), -1) # reshape image
        # print(images.shape)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += images_per_batch

    mean /= total_images 
    std /= total_images

    # approx mean and std not actual precise mean and std
    return mean, std

In [45]:
mean, std = get_mean_and_std(train_loader)
print(mean, std)

tensor([0.5285, 0.5606, 0.3673]) tensor([0.1232, 0.1068, 0.1230])


# Data Preparation

In [11]:
# Data Transformation
# no need to resize! already resized to 224 x 224 px
transform = transforms.Compose([
    transforms.ToTensor(), # convert images to pytorch tensors
    # transforms.Normalize(mean=[mean_R, mean_G, mean_B], std=[std_R, std_G, std_B])
    transforms.normalize(torch.Tensor(mean), torch.Tensor(std)) 
])

In [ ]:
class BananaDataset(Dataset):
    def __init__(self, data_dir transform=None):
        self.data_dir = data_dir
        self.transform = transform

In [ ]:
healthy_data = BananaDataset(data_dir=healthy_banana_dataset_path, transforms)
cordona_data = BananaDataset(data_dir=cordona_banana_dataset_path, transforms)
pestalotiopsis_data = BananaDataset(data_dir=pestalotiopsis_banana_dataset_path, transforms)
sigatoka_data = BananaDataset(data_dir=psigatoka_banana_dataset_path, transforms)